# Rating Scale

A Rating Scale is a standardized assesment that that reflects the creditworthiness of a borrower,Financial Instrument or and Entity which indicates its associated risk level.It is a measure of likelyhood of default or financial distress often represented in the form of letters(AAA,BB+) or Number (300-900).

To develop the rating scale of a portfolio as in the current case we need the features driving the PD (Borrower/Entity Specific Attributes),the target variable and The Predicted Probabities for the trained data as per the ML model used.In the current case ,we shall use the Logistic Regression Model.
So,First we shall Import our data from PART2 where we have stored the dataset and the PD values.

In [1]:
%store -r train_data_logistic
%store -r test_data_logistic
%store -r validation_data_logistic

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_data_logistic.shape

(594, 67)

## Master Scale

In a credit risk model, a master scale (or rating scale) serves as a standardized grid of rating grades (e.g., AAA to D or 1–10) that maps internal scores or probabilities of default (PD) to specific risk categories often categorised into bins. It provides a common language for risk across different portfolios

    We shall create a master scale that best describes the target variable according to the given parameters.

    Parameters:
    data : pandas DataFrame
        The dataset.
    default_flag : str
        The column name of the default flag(0,1).In our case it is the PD column in the train_data_logistic that contains the predicted default flag.
    PD : str
        The column name of the PD variable In our case it is the PD_p column in the train_data_logistic which indicated the probability of the default.
    bin_number : int
        The number of bins to create for binning the PD variable.Here we shall split the data into 10 bins.The choice of number of bins is coded below .
        
 -**Several rules of thumb from statistics offer a starting point for a reasonable number of bins (or bin width):**
        
    -Square-root rule: A common and simple starting point, particularly for general visualization, is to use the square root of the number of observations (N): bins = sqrt(N).
     -Sturges' formula: This formula is well-suited for normally distributed data: bins = log2(N) + 1
     .
     -Scott's normal reference rule: This method calculates an optimal bin width based on the data's standard deviation and sample size, then derives the number of bins from that width.
     -Freedman-Diaconis rule: Often considered more robust than Sturges' formula as it uses the interquartile range (IQR), making it less sensitive to outliers

 **Considerations:**
   - Assumes input data (`data`) includes numeric columns suitable for binning and analysis.
   - Provides insights into the distribution and risk assessment based on the Probability of Default (`PD`) variable  across bin categories
   - Enables customization through parameters like binning granularity (`bin_number`) for tailored risk analysis.

In [4]:
# how can we decide the value of n i.e no of bins over which the sample should be divided such that there is a proportinate distribution of 
# data in each of the bins as in the original data
# We will try Freedman Diaconis rule
import numpy as np
import pandas as pd
data=np.array(train_data_logistic['PD'])
Q1=np.percentile(data,0.25)
Q3=np.percentile(data,0.75)
IQR=Q3-Q1
IQR
# The IQR is 0 ,so it does not convey any useful information


np.float64(0.0)

In [5]:
# Sturges Method
import math
length=math.log2(len(train_data_logistic['PD']))
length
n=length +1
n
#  This method gives us the number of bins as 10.21.Hence we shall use 10 as the number of bins

10.214319120800766

In [6]:
 # Custom Binning:
# We shall implement custom binning logic on the PD variable (`PD`) of the input data (`data`) using `pd.cut`.
data=train_data_logistic
default_flag="PD"
PD="PD_p"
bin_number=10
# Implement custom binning logic here
data['PD_Bin'] = pd.cut(data[PD], bins=bin_number)# pd.cut is used to convert continuous PD_p values into 10 individual bins.This will place each of the data points in one of the 10 bins
data['PD_Bin']

455    (-0.001, 0.1]
516    (-0.001, 0.1]
147       (0.6, 0.7]
789       (0.9, 1.0]
121    (-0.001, 0.1]
           ...      
637       (0.2, 0.3]
425       (0.1, 0.2]
315       (0.9, 1.0]
225    (-0.001, 0.1]
342    (-0.001, 0.1]
Name: PD_Bin, Length: 594, dtype: category
Categories (10, interval[float64, right]): [(-0.001, 0.1] < (0.1, 0.2] < (0.2, 0.3] < (0.3, 0.4] ... (0.6, 0.7] < (0.7, 0.8] < (0.8, 0.9] < (0.9, 1.0]]

In [7]:
# Aggregate data based on PD bins(count,sum,mean,std)
summary = data.groupby('PD_Bin').agg({
    default_flag: ['count', 'sum'],
    PD: ['mean', 'std']
}).reset_index()
summary

C:\Users\siddh\AppData\Local\Temp\ipykernel_17048\3574640931.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = data.groupby('PD_Bin').agg({


PD_Bin    PD           PD_p          
                 count  sum      mean       std
0  (-0.001, 0.1]   370    0  0.006688  0.014793
1     (0.1, 0.2]    17    0  0.137085  0.026175
2     (0.2, 0.3]    12    0  0.225923  0.023082
3     (0.3, 0.4]     7    0  0.362715  0.036455
4     (0.4, 0.5]     6    0  0.469115  0.022546
5     (0.5, 0.6]     7    7  0.535657  0.034773
6     (0.6, 0.7]    10   10  0.654010  0.037396
7     (0.7, 0.8]    12   12  0.753022  0.026488
8     (0.8, 0.9]    19   19  0.858946  0.023543
9     (0.9, 1.0]   134  134  0.981224  0.023719

Calculating Weight of Evidence (WoE) involves transforming a feature's categories (bins) into numerical values, measuring their predictive power for a binary outcome (good/bad) using the natural log of the ratio of "good" to "bad" proportions:
WoE = $ ln(\frac {\text Pecentage Good in Bin}{\text Percentage Bad in Bin})$ , with steps including binning continuous variables and ensuring non-zero values. It quantifies how much a category in a bin shifts the odds towards "good" versus "bad"

In [8]:
summary[(PD, 'mean')] # This gives the mean value of PD_p column across each of the bins

0    0.006688
1    0.137085
2    0.225923
3    0.362715
4    0.469115
5    0.535657
6    0.654010
7    0.753022
8    0.858946
9    0.981224
Name: (PD_p, mean), dtype: float64

In [9]:
%store -r train_data_corr_eliminated

In [10]:
# Calculate WOE and Score using custom logic
# WOE and Score Calculation
# Computes Weight of Evidence (WOE) and Score metrics using custom logic.
# Calculates counts and distributions of 'Good' and 'Bad' observations.
# Computes Bad Rate and Probability of Default (`PD`).
# Derives Score using the formula `-ln(PD / (1 - PD)) * 100`.
def master_scale(data, default_flag, PD, bin_number):
    # Implement custom binning logic here
    data['PD_Bin'] = pd.cut(data[PD], bins=bin_number)
    # Aggregate data based on PD bins
    summary = data.groupby('PD_Bin').agg({
        default_flag: ['count', 'sum'],
        PD: ['mean', 'std']
    }).reset_index()
    summary['Total.Observations'] = summary[(default_flag, 'count')]# Total Observations is the count of the total observations in the PD column in each bin
    summary['Bad.Count'] = summary[(default_flag, 'sum')]# Since default is indicated by 1's in PD column,therefore the sum  of 1's gives the total defaults in the column in each bin
    summary['Good.Count'] = summary[(default_flag, 'count')] - summary[(default_flag, 'sum')]# Goods or 0's can be calculated by subtracting the Bad's from Total Observations in reach bin.
    
    
    summary['Good.Distr'] = summary['Good.Count'] / len(data)# If we divide the total good observation by the length of the data which is the total number 
    #of rows in the bins,We shall get the no. of good observations per row in each bin which may further represent the distribution of goods across the sample in each of the bins
    summary['Bad.Distr'] = summary['Bad.Count'] / len(data)# Similarly distribution of Bad's can be determined
    
    summary['Total.Distr'] = summary['Total.Observations'] / len(data)# Total distribution represent the information about Goods's or Bad's per row in each bin.

    summary['Bad.Rate'] = summary['Bad.Count'] / summary['Total.Observations'] # Instances of Bad's is the frequency of occurence of Bad's from the total observations per row in each bin.
    
    summary['PD'] = summary[(PD, 'mean')]# Mean of the Probability of default PD_p column in each bin.
    # WOE score is evaluated using the formula:
    summary['Score'] = -np.log(summary[(PD, 'mean')] / (1 - summary[(PD, 'mean')])) * 100
    # Select and rename columns
    woe_summary = summary[['PD_Bin', 'Total.Observations', 'Total.Distr', 'Good.Count', 'Bad.Count',
                           'Good.Distr', 'Bad.Distr', 'Bad.Rate', 'PD', 'Score']]
    woe_summary.columns = ['Final.PD.Range', 'Total.Observations', 'Total.Distr', 'Good.Count', 'Bad.Count',
                           'Good.Distr', 'Bad.Distr', 'Bad.Rate', 'PD', 'Score']

    return woe_summary

In [11]:
master_scale_data = master_scale(train_data_logistic, "default_flag", "PD_p", 10)
master_scale_data

C:\Users\siddh\AppData\Local\Temp\ipykernel_17048\2787648883.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = data.groupby('PD_Bin').agg({


,Final.PD.Range,Total.Observations,Total.Distr,Good.Count,Bad.Count,Good.Distr,Bad.Distr,Bad.Rate,PD,Score
0,"(-0.001, 0.1]",370,0.622896,370,0,0.622896,0.000000,0.000000,0.006688,500.071105
1,"(0.1, 0.2]",17,0.028620,14,3,0.023569,0.005051,0.176471,0.137085,183.971446
2,"(0.2, 0.3]",12,0.020202,8,4,0.013468,0.006734,0.333333,0.225923,123.147919
3,"(0.3, 0.4]",7,0.011785,4,3,0.006734,0.005051,0.428571,0.362715,56.359834
4,"(0.4, 0.5]",6,0.010101,3,3,0.005051,0.005051,0.500000,0.469115,12.369724
5,"(0.5, 0.6]",7,0.011785,1,6,0.001684,0.010101,0.857143,0.535657,-14.287230
6,"(0.6, 0.7]",10,0.016835,5,5,0.008418,0.008418,0.500000,0.654010,-63.671163
7,"(0.7, 0.8]",12,0.020202,5,7,0.008418,0.011785,0.583333,0.753022,-111.479456
8,"(0.8, 0.9]",19,0.031987,1,18,0.001684,0.030303,0.947368,0.858946,-180.656335
9,"(0.9, 1.0]",134,0.225589,3,131,0.005051,0.220539,0.977612,0.981224,-395.621415


In [12]:
%store master_scale_data

Stored 'master_scale_data' (DataFrame)


The WoE score is nothing but the logarithm of the  ratio of mean of probabilities of default vs mean of Prob. of non- default in each of the bins and hence represents mean probabilty of odds across each bin.
So , the scores can be calculated from the predicted probailities of default using the equation Score: log(PD/1-PD).
We insert a temporary column in the data and assign it a value 1 and then calculate the Scores from the predicted probabilites as shown:

In [13]:
def calculate_scores(data):
    data["temp"]=1
    data["Scores"]=np.log(data["PD_p"]/(data["temp"]-data["PD_p"]))
    data.drop("temp",axis =1)
    return data

In [14]:
train_data=calculate_scores(train_data_logistic)
test_data=calculate_scores(test_data_logistic)
validation_data=calculate_scores(validation_data_logistic)

In [15]:
%store train_data
%store test_data
%store validation_data

Stored 'train_data' (DataFrame)
Stored 'test_data' (DataFrame)
Stored 'validation_data' (DataFrame)


# Credit Score Scaling
 
Credit Scorecard will show how points represented by the bins generated from the probability of defaults as predicted by the Logistic Regression model. Generating the score points will involve scaling calculations from the logistic regression parameters and WoE(s) from grouped values inside each column in the woe_summary.
Formula for calculating attribute score:

Attribute Score=$ - \text ((WoE)_j * \beta_i + \frac {\text a}{\text n}) * (\text Factor) + \frac{\text Offset}{n})$

$ \text PDO = \text Factor * ln(2) $ therefore Factor = $ \frac {\text PDO}{\text ln (2)}$

$ \text Offset = \text Ceiling Score-(\text factor * \text Ln(PDO) $
Factor acts as a scaling point to convert the log of odds of an event into score points. Offset acts as a starting credit score. We can calculate Factor and Offset by the above formula

Offset = Score - Factor * ln(Odds)
Where:

    -𝑊𝑂𝐸 = weight of evidence for each grouped attribute
    -𝛽 = regression coefficient for each characteristic
    -𝑎 = intercept term from logistic regression
    -𝑛 = number of characteristics
    -𝑘 = number of groups (of attributes) in each characteristic
    -𝑝𝑑𝑜 = “points to double odds” which means  as how much points to increase for the odds of default to become half.
    -𝑂𝑑𝑑𝑠 = odds of default .In terms of PD we get odds as (1-PD)/PD
    -𝑆𝑐𝑜𝑟𝑒 = score where the odds of default is 𝑂𝑑𝑑𝑠.

** Min-Max scaling can also be used apart from factor and offset scaling**

In [16]:
import pandas as pd
import numpy as np

def scaled_score(data, PD, ceiling_score, PDO):
    """
    Create scaled scores for a given dataset.

    This function calculates scaled scores based on the provided probability of default (PD) variable
    using the provided ceiling score and PDO.

    Parameters:
    data (DataFrame): The dataset containing the PD variable.
    PD (str): The name of the probability of default (PD) variable in the dataset.
    ceiling_score (float): The ceiling score for transformation.
    PDO (float): The increase level for transformation.

    Returns:
    DataFrame: The dataset with the scaled score column added.

    """

    odds = (1 - data[PD]) / data[PD]
    factor = PDO / np.log(2)
    offset_val = ceiling_score - (factor * np.log(PDO))
    data['scaled_score'] = offset_val + factor * np.log(odds)
    return data


In [17]:
scaled_score(train_data_logistic,PD,3000,15)

,Unnamed: 0,id,date,saving_ratio,default_flag,max_dpd_6_months,problematic_cheque_count,duration_in_month,credit_amount,installment_as_income_perc,...,business_type_mean_encoded,other_debtors_mean_encoded,purpose_mean_encoded,sector_risk_capped_0.99_woe_encoded,PD,PD_p,PD_Bin,temp,Scores,scaled_score
455,455,456,2020-12-20,0.46,0,0,3,24,2679,4,...,0.260870,0.29989,0.165049,-0.001891,0,8.263675e-05,"(-0.001, 0.1]",1,-9.400973,3144.837708
516,516,517,2022-09-27,0.04,0,0,0,6,1361,2,...,0.260870,0.29989,0.380342,-0.001891,0,3.134459e-07,"(-0.001, 0.1]",1,-14.975639,3265.475839
147,147,148,2021-04-16,0.50,0,0,0,12,682,4,...,0.260870,0.29989,0.380342,-0.001891,1,6.117294e-01,"(0.6, 0.7]",1,0.454587,2931.559176
789,789,790,2022-05-13,0.67,1,0,2,40,5998,4,...,0.260870,0.29989,0.440000,-0.001891,1,9.818808e-01,"(0.9, 1.0]",1,3.992497,2854.997308
121,121,122,2020-03-19,0.22,0,0,1,24,3868,4,...,0.391061,0.29989,0.165049,-0.001891,0,4.651219e-05,"(-0.001, 0.1]",1,-9.975750,3157.276110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,637,638,2022-06-10,0.21,0,0,1,60,15653,2,...,0.260870,0.29989,0.221429,-0.001891,0,2.117288e-01,"(0.2, 0.3]",1,-1.314536,2969.843762
425,425,426,2020-05-16,0.23,0,0,0,18,2779,1,...,0.391061,0.29989,0.165049,-0.001891,0,1.192047e-01,"(0.1, 0.2]",1,-1.999983,2984.677117
315,315,316,2023-04-25,0.86,1,0,1,36,2746,4,...,0.260870,0.29989,0.320442,-0.001891,1,9.961742e-01,"(0.9, 1.0]",1,5.562167,2821.028975
225,225,226,2021-04-28,0.31,0,0,0,36,2613,4,...,0.260870,0.29989,0.363636,-0.001891,0,2.013052e-04,"(-0.001, 0.1]",1,-8.510487,3125.567201


In [18]:
# As we can observe above that the scaled score surpasses the ceiling score and hence needs calibration

# Bayesian Calibration
**Priors:** Existing knowledge of model coefficients against probability of default values often known as Probability Distributions.

**Data Collection:** New Data Point acts as the basis of predicting the likelyhood of default for given model parameters.(central_tendency)

**Bayes Inference:** Using Bayes' Theorem, the prior probabilities are combined with the likelihood from the data to produce posterior probabilities that aling each of the individual values in PD to ajust such that the average value of PD matches that of the model parameter (central_tendency)
Instead of single point estimates, Bayesian methods yield probability distributions for model parameters, reflecting uncertainty.

**Calibration:** The process adjusts the model's output (like default probabilities) to align with observed outcomes, often using techniques like Expected Maximization (EM) or optimizing scoring rules to match true posterior distributions.

**Self-Learning:** As new loan data comes in, the model updates its posterior distributions, allowing it to adapt and improve its predictions over time without needing complete retraining

# Key Advantages

    -Incorporates Prior Info: Banks can use historical insights or regulatory guidance, improving models even with limited new data.
    -Handles Uncertainty: Provides a full probability distribution for risk, not just a single number, better reflecting true risk.
    -Data-Driven & Objective: Leverages bank's own data for bespoke models, reducing reliance on subjective human judgment.
    -Improved Performance: Often outperforms traditional single-stage methods (like standard logistic regression) in prediction accuracy (e.g., higher KS, AUC).
    -Self-Learning: Adapts to evolving economic conditions and customer behaviors. 

**Mechanism:** It typically scales a "Point-in-Time" (PIT) probability to a "Through-the-Cycle" (TTC) level using a scaling factor derived from the portfolio's central tendency.
Example (Conceptual):
Suppose a rating grade has a prior expected PD of 1% (from 10 years of data). In the current year, only 2 defaults are observed out of 500 loans (0.4%). Bayesian methods blend these to find a posterior PD (e.g., 0.8%), preventing drastic model shifts due to one "lucky" year.

The Calibration involves adjusting the PD looking at the pre calculated target i.e central_tendecy of the portfolio.
The step by step process flow is coded as below:-

Implementing Bayesian calibration for scaled credit scores involves two main phases: Mapping scores back to Point-in-Time (PIT) probabilities and Adjusting those probabilities to a Through-the-Cycle (TTC) target using a Bayesian scaling factor.

To simplify this process, we break the Bayesian calibration into four logical blocks. This method adjusts  existing scores (already scaled via factor and offset) to ensure the final Probability of Default (PD) aligns with a target Central Tendency (the long-run average default rate expected by the bank or regulator).

Block 1: Calculating Portfolio-Wide Observed PD
Before calibrating, we must find the current average PD of your master scale that has already been calculated.

In [19]:
master_scale_data

,Final.PD.Range,Total.Observations,Total.Distr,Good.Count,Bad.Count,Good.Distr,Bad.Distr,Bad.Rate,PD,Score
0,"(-0.001, 0.1]",370,0.622896,370,0,0.622896,0.000000,0.000000,0.006688,500.071105
1,"(0.1, 0.2]",17,0.028620,14,3,0.023569,0.005051,0.176471,0.137085,183.971446
2,"(0.2, 0.3]",12,0.020202,8,4,0.013468,0.006734,0.333333,0.225923,123.147919
3,"(0.3, 0.4]",7,0.011785,4,3,0.006734,0.005051,0.428571,0.362715,56.359834
4,"(0.4, 0.5]",6,0.010101,3,3,0.005051,0.005051,0.500000,0.469115,12.369724
5,"(0.5, 0.6]",7,0.011785,1,6,0.001684,0.010101,0.857143,0.535657,-14.287230
6,"(0.6, 0.7]",10,0.016835,5,5,0.008418,0.008418,0.500000,0.654010,-63.671163
7,"(0.7, 0.8]",12,0.020202,5,7,0.008418,0.011785,0.583333,0.753022,-111.479456
8,"(0.8, 0.9]",19,0.031987,1,18,0.001684,0.030303,0.947368,0.858946,-180.656335
9,"(0.9, 1.0]",134,0.225589,3,131,0.005051,0.220539,0.977612,0.981224,-395.621415


In [20]:
import statsmodels.api as sm
import numpy as np
# 1. Calculate the weighted average PD of the current master scale
total_obs = master_scale_data["Total.Observations"].sum()
total_defaults = (master_scale_data["Total.Observations"] * master_scale_data["PD"]).sum()
avg_pd = total_defaults / total_obs
central_tendency=0.5
# This is the "Point-in-Time" (PIT) average. We multiply the PD of each grade(bin) by the number of observations in that grade to find the total expected defaults, 
# then divide by the total population to get the average pd for that bin.

Block 2: The Bayesian Shift (The "Scales" Adjustment)
This is the core Bayesian step. We adjust the individual PDs of each rating grade(bin) so that the entire portfolio's average shifts to match the central_tendency.

In [21]:
# 2. Adjust the individual PDs for each observation using the Bayesian formula (Scales Shift)
# Prior: central_tendency / avg_pd
# Likelihood: The existing PD
numerator = (master_scale_data["PD"] * central_tendency / avg_pd)
denominator = numerator + (1 - master_scale_data["PD"]) * ((1 - central_tendency) / (1 - avg_pd))

master_scale_data['calibrated_pd'] = numerator / denominator
# The central_tendency variable in your code acts as the "anchor". If your current avg_pd (calculated from your model) is 2% but your historical central
#_tendency is 3%, the Bayesian calibration will shift all individual PDs upward to ensure the final portfolio average matches that 3% target
# 3. Convert calibrated PD to Log-Odds and find the linear relationship
# Calculate the target Odds Ratio
master_scale_data['OddRatio'] = (1 - master_scale_data['calibrated_pd']) / master_scale_data['calibrated_pd']

# Reset index to ensure a clean 0 to N range for both
master_scale_data = master_scale_data.reset_index(drop=True)

# Define X and y from the same cleaned DataFrame
X = sm.add_constant(master_scale_data["Score"])
y = np.log(master_scale_data['OddRatio']) # Ensure this column is in master_scale_data

OLS_model = sm.OLS(y, X).fit()
intercept = OLS_model.params.iloc[0]
score_coef = OLS_model.params.iloc[1]
#  The relationship between a score and PD is non-linear (logistic). 
# By taking the log of the odds, we create a linear target. 
# The Regression finds the Intercept and Coefficient that best maps your scaled scores to these new Bayesian-adjusted probabilities.
# 4. Map the formula back to the individual scores
# Formula: PD = 1 / (1 + exp(Intercept + Score * Coef))
linear_component = intercept +(train_data_logistic[["scaled_score"]] * score_coef)
train_data_logistic['calibrated_pd'] = 1 / (1 + np.exp(linear_component))
# This is the Logistic Sigmoid Function. It converts the linear output of the regression back into a probability between 0 and 1. 
# This ensures that every customer score now results in a PD that is consistent with the bank's required Central Tendency.

In [22]:
# 4. Map the formula back to the individual scores
# Formula: PD = 1 / (1 + exp(Intercept + Score * Coef))
linear_component = intercept +(train_data_logistic[["scaled_score"]] * score_coef)
train_data_logistic['calibrated_pd'] = 1 / (1 + np.exp(linear_component))
# This is the Logistic Sigmoid Function. It converts the linear output of the regression back into a probability between 0 and 1. 
# This ensures that every customer score now results in a PD that is consistent with the bank's required Central Tendency.

In [23]:
# Ensure parameters from previous steps are ready
intercept = OLS_model.params.iloc[0]
score_coef = OLS_model.params.iloc[1]
calibration_data=train_data_logistic
# 4. Map the formula back to individual customer records
# Linear Component (z) = Intercept + (Score * Coefficient)
linear_component = intercept + (train_data_logistic["scaled_score"] * score_coef)

# Logistic Sigmoid Function: 1 / (1 + exp(z))
# This converts the linear log-odds into a valid PD (0 to 1)
calibration_data['calibrated_pd'] = 1 / (1 + np.exp(linear_component))

# Verify the results
print(calibration_data[['scaled_score', 'calibrated_pd']].head())


     scaled_score  calibrated_pd
455   3144.837708   5.056757e-14
516   3265.475839   1.513378e-14
147   2931.559176   4.267062e-13
789   2854.997308   9.175567e-13
121   3157.276110   4.465322e-14


In [33]:
calibration_data.head()

,default_flag,Net_Profit_Margin,ROA,sector_risk,ROE,saving_ratio,modelpd,modelscore,calibrated_pd,calibrated_score
22,0,-9.0,35,3,-8,0.12,1.966126e-08,-17.744616,1.282577e-10,-22.776980
26,0,23.0,26,0,67,0.12,1.041562e-06,-13.774788,1.952784e-08,-17.751425
32,0,62.0,20,2,37,0.07,1.674918e-04,-8.694409,1.212804e-05,-11.319978
46,0,43.0,48,1,94,0.46,4.827540e-04,-7.635520,4.633847e-05,-9.979492
58,0,-5.0,35,1,25,0.49,6.585305e-08,-16.535840,5.924558e-10,-21.246745


In [34]:
%store calibration_data

Stored 'calibration_data' (DataFrame)


In [24]:
print(OLS_model.summary())

                            OLS Regression Results                            
Dep. Variable:               OddRatio   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.124e+31
Date:                Wed, 21 Jan 2026   Prob (F-statistic):          1.18e-123
Time:                        17:12:23   Log-Likelihood:                 329.89
No. Observations:                  10   AIC:                            -655.8
Df Residuals:                       8   BIC:                            -655.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8329   4.03e-16  -2.07e+15      0.0

In [25]:
feature_names = OLS_model.model.exog_names
print(feature_names)

['const', 'Score']


In [26]:
# Returns the name of the target variable (y)
target_name = OLS_model.model.endog_names
print(target_name)

OddRatio


In credit risk calibration, OLS regression is a tool to find the link between Scores and Log-Odds. Interpreting the results involves looking at the parameters that define the calibration curve and the metrics that prove its reliability. 
1. The Intercept and Coefficient (The Calibration Parameters) These values are the "DNA" of your calibration formula: $\text (LogOdds=Intercept+(Score * Coefficient)$).
2. Intercept: This represents the baseline log-odds when the score is zero. In practical credit risk terms, it is a mathematical anchor that positions the PD curve vertically to ensure the portfolio's average PD matches your Central Tendency.Score
3. Coefficient: This represents the sensitivity of risk to the score.Direction: A negative coefficient is standard for credit scores (where higher scores = lower risk). It means for every 1-unit increase in score, the log-odds of default decrease.Magnitude: This tells you the rate of change. If the coefficient is -0.05, a 100-point increase in score results in a 5-unit drop in log-odds.
4. P-Values and Significance P > |t|: This is the most critical diagnostic. You want a p-value less than 0.05 for your score coefficient.Meaning: A p-value < 0.05 indicates that your Score is a statistically significant predictor of the target PD. If it is higher, your model likely lacks "calibration power"—meaning the score doesn't reliably distinguish between different levels of risk.
5. R-Squared (Goodness of Fit) : Measures the proportion of variance in the log-odds explained by your scores.Credit Risk Context: In calibration (unlike initial model building), we often see very high $ (\text R^{2})$ (0.90+) because the master scale is already smoothed and rank-ordered. A low $ (\text R^{2})$ here suggests that your "scaled scores" are not linearly related to the log-odds, potentially requiring a different calibration method (like Isotonic Regression).
6. Diagnostic Metrics for Models F-Statistic: Tests the overall significance of the model. A high F-statistic (and low Prob F-stat) confirms that the entire regression is better than a random guess.
7. Durbin-Watson: Should be close to 2.0. This confirms there is no "autocorrelation" in your rating grades—meaning the risk jump between Grade 1 and Grade 2 is independent of the jump between Grade 2 and Grade 3.
8. Confidence Intervals ([0.025, 0.975]): For a stable model, these intervals should be narrow. If they are very wide, your calibration may be unstable and could shift drastically with small changes in the data. 

Since we have already obtained the scores and hence the probability distribution in the form of calibrated_PD aligned with model parameters which are predetermined either through historic data or through model governance,we can derive the default flag variable for each of these scores.
To derive the default flag we would calculate the Probabilities for each of the scores and these probabilities can be further converted into 1's and 0's.
Regressing the calibrated score with the default flag to get the calibrated_PD= $\frac {1}{1+e^(-CalibratedScore)}$ 
Implementation over the calibrated_PD is done as coded below:

In [27]:
%store -r model # The Logistic Regression model trained on the data stored in part 2 is imported 

no stored variable or alias #
no stored variable or alias The
no stored variable or alias Logistic
no stored variable or alias Regression
no stored variable or alias trained
no stored variable or alias on
no stored variable or alias the
no stored variable or alias data
no stored variable or alias stored
no stored variable or alias in
no stored variable or alias part
no stored variable or alias 2
no stored variable or alias is
no stored variable or alias imported


In [28]:
model.feature_names_in_# These are the set of features over which the LR model was trained

array(['Net_Profit_Margin', 'ROA', 'sector_risk', 'ROE', 'saving_ratio'],
      dtype=object)

In [29]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression


def regression_calibration(model, calibration_data, default_flag):
    # Calculate model predicted probabilities
    calibration_data['modelpd'] = model.predict_proba(calibration_data.drop(default_flag, axis=1))[:, 1] # The trained LR model on traing data is used to predict the PD and stored as model_pd on the calibration data

    # Calculate model scores
    calibration_data['modelscore'] = np.log(calibration_data['modelpd'] / (1 - calibration_data['modelpd'])) # Convert the [predicted PD values in calibration data into log odds or modelscores

    # Fit calibration model
    calibration_model = LogisticRegression()
    calibration_model.fit(calibration_data[['modelscore']], calibration_data[default_flag]) # fit the calibration model according to the modelscores as calculated from the above step

    # Calculate calibrated probabilities
    calibration_data['calibrated_pd'] = calibration_model.predict_proba(calibration_data[['modelscore']])[:, 1] # Predict the PD for the calculated modelscores

    # Calculate calibrated scores
    calibration_data['calibrated_score'] = np.log(calibration_data['calibrated_pd'] / (1 - calibration_data['calibrated_pd']))# convert the predicted PDs as log odds as the calinrated score

    # Prepare formula strings
    var_names = calibration_data.columns[:-4]  # Exclude modelpd, modelscore, calibrated_pd, calibrated_score
    equation_str = " + ".join([f"{coef:.7f}*{var_name}" for coef, var_name in zip(model.coef_[0], var_names)])
    equation_str = f"{equation_str} + {model.intercept_[0]}"
    equation_str = f"{model.coef_[0]}*{var_names} + {model.intercept_[0]}"

    var_names_cal = calibration_data.columns[:-6]  # Exclude modelpd, modelscore, calibrated_pd, calibrated_score, default_flag
    equation_str_cal = " + ".join([f"{coef:.7f}*{var_name}" for coef, var_name in zip(calibration_model.coef_[0], var_names_cal)])
    equation_str_cal = f"{equation_str_cal} + {calibration_model.intercept_[0]}"

    # Prepare calibration formula
    calibration_formula = f"modelscore formula ::: {equation_str}, calibrated_score formula ::: {equation_str_cal}, Calibration Formula to get calibrated_pd ::: 1/(1 + exp(-calibrated_score))"

    return calibration_data,calibration_model,calibration_formula


In [30]:
calibration_data, calibration_model, calibration_formula = regression_calibration(
    model, 
    validation_data_logistic[['default_flag','Net_Profit_Margin', 'ROA', 'sector_risk', 'ROE', 'saving_ratio']], 
    "default_flag"
)
print(calibration_formula)
print(calibration_model)

modelscore formula ::: [0.13210864 0.0554701  0.61455298 0.02780321 2.68092172]*Index(['default_flag', 'Net_Profit_Margin', 'ROA', 'sector_risk', 'ROE',
       'saving_ratio'],
      dtype='object') + -20.44003516377127, calibrated_score formula ::: 1.2659380*default_flag + -0.3133958970876711, Calibration Formula to get calibrated_pd ::: 1/(1 + exp(-calibrated_score))
LogisticRegression()


C:\Users\siddh\AppData\Local\Temp\ipykernel_17048\2735133407.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calibration_data['modelpd'] = model.predict_proba(calibration_data.drop(default_flag, axis=1))[:, 1] # The trained LR model on traing data is used to predict the PD and stored as model_pd on the calibration data
C:\Users\siddh\AppData\Local\Temp\ipykernel_17048\2735133407.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calibration_data['modelscore'] = np.log(calibration_data['modelpd'] / (1 

In [31]:
calibration_data.head()

,default_flag,Net_Profit_Margin,ROA,sector_risk,ROE,saving_ratio,modelpd,modelscore,calibrated_pd,calibrated_score
22,0,-9.0,35,3,-8,0.12,1.966126e-08,-17.744616,1.282577e-10,-22.776980
26,0,23.0,26,0,67,0.12,1.041562e-06,-13.774788,1.952784e-08,-17.751425
32,0,62.0,20,2,37,0.07,1.674918e-04,-8.694409,1.212804e-05,-11.319978
46,0,43.0,48,1,94,0.46,4.827540e-04,-7.635520,4.633847e-05,-9.979492
58,0,-5.0,35,1,25,0.49,6.585305e-08,-16.535840,5.924558e-10,-21.246745


In [32]:
%store calibration_data

Stored 'calibration_data' (DataFrame)
